In [4]:
from datasets import load_dataset

dataset = load_dataset("lst20", data_dir="./khmernlp/dataset/x.tart.gz")

d:\miniconda3\envs\khmernlp\lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for lst20 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/lst20
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


FileNotFoundError: d:\code\projects\khmernlp\khmernlp\dataset\x.tart.gz does not exist. Make sure you insert a manual dir via `datasetts.load_dataset('lst20', data_dir=...)`. Manual download instructions:   You need to
  1. Manually download `AIFORTHAI-LST20Corpus.tar.gz` from https://aiforthai.in.th/corpus.php (login required; website mostly in Thai)
  2. Extract the .tar.gz; this will result in folder `LST20Corpus`
  The <path/to/folder> can e.g. be `~/Downloads/LST20Corpus`.
  lst20 can then be loaded using the following command `datasets.load_dataset("lst20", data_dir="<path/to/folder>")`.
  )

In [2]:
import re
from collections import defaultdict
from heapq import heappush, heappop  # for priority queue
# from marisa_trie import Trie

from khmernlp.tokenize import DEFAULT_WORD_DICT_TRIE




# wordlist = [li.strip() for li in open('./khmer_dictionary.txt', 'r', encoding='utf-8')]
# trie = Trie(wordlist)

trie = DEFAULT_WORD_DICT_TRIE

# ช่วยตัดพวกภาษาอังกฤษ เป็นต้น
pat_eng = re.compile(r'''(?x)
[-a-zA-Z]+|   # english
\d[\d,\.]*|   # number
[ \t]+|       # space
\r?\n         # newline
''')



def onecut(text):
  words_at = defaultdict(list)  # main data structure

  def serialize(p, p2):    # helper function
    for w in words_at[p]:
      p_ = p + len(w)
      if p_== p2:
        yield [w]
      elif p_ < p2:
        for path in serialize(p_, p2):
          yield [w]+path

  q = [0]       # min-heap queue
  last_p = 0    # last position for yield
  while q[0] < len(text):
      p = heappop(q)

      for w in trie.prefixes(text[p:]):
          words_at[p].append(w)
          if p+len(w) not in q:
            heappush(q, p+len(w))

      if len(q)==1:
          for w in min(serialize(last_p, q[0]), key=len):
            yield w
          last_p = q[0]

      # กรณี len(q) == 0  คือ ไม่มีใน dict
      if len(q)==0:
          m = pat_eng.match(text[p:])
          if m: # อังกฤษ, เลข, ว่าง
              i = p + m.span()[1]
          else: # skip น้อยที่สุด ที่เป็นไปได้
              for i in range(p, len(text)):
                  ww = trie.prefixes(text[i:])
                  m = pat_eng.match(text[i:])
                  if ww or m:
                      break
              else:
                  i = len(text)
          w = text[p:i]
          words_at[p].append(w)
          yield w
          last_p = i
          heappush(q, i)

ImportError: cannot import name 'path_pythainlp_corpus' from 'khmernlp.corpus.core' (/Users/macbookair/code/projects/khmerASR/pythainlp/khmernlp/corpus/core.py)